# Programacion Concurrente - TP2 - Parte 1

Utilizamos 2 prácticas con OpenMP desarrolladas en lenguaje C/C++ nativo en plataforma colab. 


---
## 1.1. Ejercicio 1 - Hola Mundo estilo Programacion Concurrente


### 1.1.1. Preguntas ejercicio 1:

a) Identifique los 3 componentes de openMP en el ejercicio 1.

b) Realice pruebas modificando la cantidad de hilos (1, 3, 7). Que conclusiones pudo determinar?.

c) ¿Cuál es la diferencia las formas de asignación *static* y *dynamic* en la cláusula *schedule*?, ¿Qué sucede si utiliza *dynamic* en el código?

d) En el for: ¿Que sucede con el valor de j, sí se parametriza como lastprivate?


### 1.1.2. Código Lenguaje C 

In [ ]:
%%writefile code.cpp

// Hola Mundo desde OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <omp.h>    // Cabecera OpenMP   
  
int main(int argc, char* argv[]) 
{
  int i,j=0;
  std::cout<<"Inicio..."<<std::endl;

  //-----------------------------------------------------------------------------------------------
  // Region paralela
  #pragma omp parallel
  { 
    std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", de "<<  omp_get_num_threads() <<", procesadores "<< omp_get_num_procs()<< std::endl;  

    #pragma omp single
    {
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< " uno de muchos."<< std::endl;
    }
    #pragma omp master
    {
      std::cout<<"Hola mundo!!!... soy el hilo maestro: " << omp_get_thread_num()<< std::endl;
    }

    #pragma omp for schedule(dynamic, 5)
    for(i=0;i<20;i++)
    {
      j++;
      std::cout<<"Hola mundo!!!... soy el hilo " << omp_get_thread_num()<< ", itero para i ."<<i<<", actualizo j "<<j<< std::endl;
    }
  }
  // Region paralela
  //-----------------------------------------------------------------------------------------------

  std::cout<<"Fin..., con j="<<j<<std::endl;
}

### 1.1.3 Compilación de Hola Mundo. 

In [ ]:
!g++ -o hello -fopenmp code.cpp

### 1.1.4. Ejecución Hola Mundo en OpenMP.

In [ ]:
%env OMP_NUM_THREADS=5
!./hello

---
## 1.2. Ejercicio 2 - Axpy

La función Axpy es parte de las bibliotecas BLAS [3]. Ella se encarga de realizar la suma de vectores, con la siguiente ecuación:

<center>$Y = \alpha X + Y$</center>

En donde:
> $\alpha$: Es un escala.

> $X$: Es el vector origen.

> $Y$: Es el vector destino.


### 1.2.1 Preguntas Ejercicio 2
a) Ejecute el ejercicio 1.2.2:

    - Con la variable cantidad_N para: 50, 500, 5.000.
    - Con los valores hilos_ 2, 4, 6.

b) Repita la prueba (a), con ciclos 1, 10, 30. ¿Cuál fue la diferencia?

c) ¿Por qué el SpeedUp no mejora a medida que aumentan la cantidad de hilos?

d) ¿Qué sucede con la eficiencia a medida que aumentan la cantidad_N?¿Porqué no llega al valor ideal?.

### 1.2.2 Código Lenguaje C 

In [ ]:
%%writefile code_axpy.cpp
// Axpy con OpenMP, usando c, ejecutado en Colab. 

#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP   

// ----------------------------------------------------------------------------
// Macros que miden el tiempo.

static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = tv.tv_sec + tv.tv_usec/1000000.0;
   
   
#define TIEMPO_FIN( h )      \
   gettimeofday(&tv,NULL);   \
   dHashTiempoHistory[ h ] = ((tv.tv_sec + tv.tv_usec/1000000.0) - dHashTiempoHistory[ h ]) * 1000; // Devuelvo en milisegundos
#define TIEMPO_GET( h ) dHashTiempoHistory[ h ]

#define HTH_TOTAL         1
#define HTH_AXPY_SEC      2
#define HTH_AXPY_OMP      3

using namespace std;

// ----------------------------------------------------------------------------

void axpy( double alfa, vector<double> &X, vector<double> &Y )
{
  int i;

  #pragma omp parallel for
  for(i=0;i<Y.size();i++)
  {
    Y[i] = alfa * X[i] + Y[i];
  }
}

// ----------------------------------------------------------------------------

int main(int argc, char* argv[]) 
{ 
  int i,c;
  TIEMPO_INI( HTH_TOTAL )

  // Leo los parametros.
  if( argc != 4 )
  {
      std::cerr<< " Error en los parametros de indicar: (alfa), (Tamanio vector), (ciclos ejecucion)."<<argc<<std::endl;
      exit( -1 );
  }

  float alfa     = atof( argv[1] );
  int cantidad_N = atoi( argv[2] );
  int ciclos     = atoi( argv[3] );

  // --------------------------------------------
  // Defino la memoria de los vectores.

  vector<double> X( cantidad_N );
  vector<double> Y( cantidad_N );

  for (int i=0;i<X.size();i++)
  {
    X[i] = (rand()/(double)RAND_MAX)*0.73;
    Y[i] = (rand()/(double)RAND_MAX)*0.71;
  }

  // --------------------------------------------
  // Realizo la función Axpy con OpenMP.

  TIEMPO_INI( HTH_AXPY_OMP )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_OMP )

  // --------------------------------------------
  // Realizo la función Axpy en forma secuencial.

  omp_set_num_threads(1);

  TIEMPO_INI( HTH_AXPY_SEC )

  for(c=0;c<ciclos;c++)
  {
    axpy( alfa, X, Y );
  }

  TIEMPO_FIN( HTH_AXPY_SEC )

  TIEMPO_FIN( HTH_TOTAL )

 std::cout<<"Valores Reales  :" <<std::endl;
 std::cout<<"Tiempo TOTAL     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;
 std::cout<<std::endl;
 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


 std::cout<<std::endl;
 std::cout<<"Valores Ideal: "<<std::endl;
 TIEMPO_GET(HTH_AXPY_OMP) = TIEMPO_GET(HTH_AXPY_SEC) / omp_get_num_procs();
 std::cout<<"Tiempo axpy Sec  : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" [ms]"<<std::endl;
 std::cout<<"Tiempo axpy Omp  : "<<TIEMPO_GET(HTH_AXPY_OMP)<<" [ms]"<<std::endl;

 std::cout<<"SpeedUp          : (tiempo Secuencial/tiempo paralelo) : "<<TIEMPO_GET(HTH_AXPY_SEC)<<" / "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Eficiencia       : SpeedUp/nro procesadores            : "<<TIEMPO_GET(HTH_AXPY_SEC)/TIEMPO_GET(HTH_AXPY_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))<<std::endl;
 std::cout<<"Coste Sec        : nro procesadores*Tiempo             : "<<1<<" * "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;
 std::cout<<"Coste Omp        : nro procesadores*Tiempo             : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_AXPY_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<std::endl;
 std::cout<<"Funcion Overhead : Coste Omp - tiempo Secuencial       : "<<omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP)<<" - "<<TIEMPO_GET(HTH_AXPY_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_AXPY_OMP))-TIEMPO_GET(HTH_AXPY_SEC)<<std::endl;


}
// ----------------------------------------------------------------------------

### 1.2.3. Compilación de código C Axpy. 

In [ ]:
!g++ -o axpy -fopenmp code_axpy.cpp

### 1.2.4. Ejecución Axpy.

In [ ]:
# --------------------------------------------
#@title Parámetros de ejecución { vertical-output: true }

Cantidad_N = 50#@param {type: "number"}
Alfa       = 1.0#@param {type: "number"}
Ciclos     = 10#@param {type: "slider", min: 10, max: 200}
Hilos      = 2#@param {type: "slider", min: 1, max: 10}

# --------------------------------------------

%env OMP_NUM_THREADS=$Hilos
!./axpy $Alfa $Cantidad_N $Ciclos

---
# Medidas de prestaciones en algoritmos paralelos
Las técnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como medida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [6,7]. 


## SpeedUp
Referencia a la ganancia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = \frac{Tiempo\_Secuencial}{Tiempo\_Paralelo} $</center>



##Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganancia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = \frac{SpeedUp}{Nro\_Procesadores} $</center>


## Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resolución del problema.

<center>$ coste = Nro\_Procesadores \times Tiempo\_Algoritmo$</center>


## Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.

<center>$ Overhead = {Coste}-{Tiempo\_Secuencial} $</center>

---

In [ ]:
!cat /proc/cpuinfo

---
# Bibliografía
[1] Función Axpy de biblioteca BLAS: [Referencia](https://software.intel.com/content/www/us/en/develop/documentation/mkl-developer-reference-c/top/blas-and-sparse-blas-routines/blas-routines/blas-level-1-routines-and-functions/cblas-axpy.html)

[3] Biblioteca BLAS: [Referencia](http://www.netlib.org/blas/)

[4] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[5] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://progc-unlam.com.ar/material-clase/OpenMP-MPI/Introducci%C3%B3n%20a%20la%20Computaci%C3%B3n%20Paralela.pdf)